In [38]:
import cv2 as cv
import rasterio as rio
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import glob

In [78]:
inputFolder = input(r'Input the images folder name: ')
fileList = glob.glob(f"{inputFolder}\*.tif")

In [79]:
len(fileList)

28

In [80]:
# Get image size:
with rio.open(fileList[0]) as dst:
    tempImg = dst.read()
    imgShape = tempImg.shape
    imgMeta = dst.meta
    del tempImg

In [62]:
imgShape

(4, 1073, 1080)

In [82]:
stackImg = np.zeros((imgShape[1], imgShape[2], 4*len(fileList)))

In [ ]:
i = 0
j = 0
while i<len(fileList):
    with rio.open(fileList[i]) as dst:
        inpImg = np.dstack((dst.read(1), dst.read(2), dst.read(3), dst.read(4)))
        stackImg[:,:,j:j+4]=inpImg
        j+=4
        i+=1
    print(i,j)

In [28]:
i,j

(32, 128)

In [84]:
inpImg.shape

(1073, 1080, 4)

In [85]:
depth = stackImg.shape[2]

In [86]:
satz = stackImg.reshape((-1, depth))

In [87]:
satz = np.float32(satz)

In [88]:
criteria = (cv.TERM_CRITERIA_EPS + 3, 10, 1.0)

In [89]:
k = 50
ret,label,center=cv.kmeans(satz,k,None,criteria,10,cv.KMEANS_RANDOM_CENTERS)

In [90]:
label.shape

(1158840, 1)

In [91]:
label+= 1


In [92]:
labelImg = label.reshape(imgShape[1:])

In [93]:
outFolder = input(r"Name of output folder: ")

In [94]:
outFpat = Path(outFolder, "kmeans_cluster_allImgages_2.tif")

In [95]:
labelImg = np.uint16(labelImg)

In [75]:
imgMeta

{'driver': 'GTiff',
 'dtype': 'uint16',
 'nodata': 0.0,
 'width': 1080,
 'height': 1073,
 'count': 4,
 'crs': CRS.from_epsg(32644),
 'transform': Affine(10.0, 0.0, 278160.0,
        0.0, -10.0, 2031040.0)}

In [96]:
imgMeta.update(count = 1)

In [97]:
with rio.open(outFpat, 'w', **imgMeta) as dst:
    dst.write(labelImg, 1)